Notebook Setup

In [2]:
#import dependencies
import requests
from pprint import pprint
from requests.auth import HTTPBasicAuth
import pandas as pd

#authentication flow
client_id = "4b454f446d1f479082eec9de1c27a4a8"
client_secret = "e639450422594948b17c46d1a82f4882"

url = "https://accounts.spotify.com/api/token?grant_type=client_credentials"
headers = {'content-type': 'application/x-www-form-urlencoded'}

try:
   response = requests.post(url, headers=headers, auth=(client_id, client_secret))
   json = response.json()
   print(json)
   access_token = json["access_token"]
   print(access_token)
except:
   print("Something went wrong with the request")

{'access_token': 'BQDUAULT5_hYbYolqFY3rtg51y16VYDDsKxUkO_ApSdMBzl2tVW6j3DpQY3AyJVpoXqNhkwhpm8oZI10oJQ', 'token_type': 'Bearer', 'expires_in': 3600, 'scope': ''}
BQDUAULT5_hYbYolqFY3rtg51y16VYDDsKxUkO_ApSdMBzl2tVW6j3DpQY3AyJVpoXqNhkwhpm8oZI10oJQ


Collect Data

In [4]:
# https://api.spotify.com/v1/artists/{id}

#This is for US Top 50 playlist
playlist_id = "37i9dQZEVXbLRQDuF5jeBp"
#spotify:user:spotifycharts:playlist:37i9dQZEVXbLRQDuF5jeBp

url = f"https://api.spotify.com/v1/playlists/{playlist_id}"

headers = {'authorization': f"Bearer {access_token}"}
response = requests.get(url, headers=headers)

#pprint(response.json())
#print(json.dumps(newrelease, sort_keys=True, indent=4))
jj = response.json()
#pprint(jj)

In [5]:
#test check: looking for track name of first song
pprint(jj['tracks']['items'][0]['track']['name'])

'7 rings'


In [40]:
songs= jj['tracks']['items']

In [8]:
#Create lists for relevant data & then organize into a dataframe
ids = []
artists = []
tracks = []
audiofeatures = []
for x in songs:
    idsong = x["track"]["id"]
    ids.append(idsong)
    artist = x['track']['album']['artists'][0]['name']
    artists.append(artist)
    track = x['track']['name']
    tracks.append(track)
    url = f"https://api.spotify.com/v1/audio-features/{idsong}"
    headers = {'authorization': f"Bearer {access_token}"}
    response = requests.get(url, headers=headers)
    features= response.json()
    audiofeatures.append(features)


In [9]:
pprint(audiofeatures)

[{'acousticness': 0.601,
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/6ocbgoVGwYJhOv1GgI9NsF',
  'danceability': 0.771,
  'duration_ms': 178627,
  'energy': 0.32,
  'id': '6ocbgoVGwYJhOv1GgI9NsF',
  'instrumentalness': 0,
  'key': 1,
  'liveness': 0.0885,
  'loudness': -10.752,
  'mode': 0,
  'speechiness': 0.316,
  'tempo': 139.844,
  'time_signature': 4,
  'track_href': 'https://api.spotify.com/v1/tracks/6ocbgoVGwYJhOv1GgI9NsF',
  'type': 'audio_features',
  'uri': 'spotify:track:6ocbgoVGwYJhOv1GgI9NsF',
  'valence': 0.312},
 {'acousticness': 0.0378,
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/4kV4N9D1iKVxx1KLvtTpjS',
  'danceability': 0.726,
  'duration_ms': 190440,
  'energy': 0.545,
  'id': '4kV4N9D1iKVxx1KLvtTpjS',
  'instrumentalness': 0,
  'key': 5,
  'liveness': 0.108,
  'loudness': -5.489,
  'mode': 0,
  'speechiness': 0.102,
  'tempo': 169.991,
  'time_signature': 4,
  'track_href': 'https://api.spotify.com/v1/tracks/4kV4N9D1iKVxx1KLvtTpjS',


  'tempo': 89.949,
  'time_signature': 4,
  'track_href': 'https://api.spotify.com/v1/tracks/7FGq80cy8juXBCD2nrqdWU',
  'type': 'audio_features',
  'uri': 'spotify:track:7FGq80cy8juXBCD2nrqdWU',
  'valence': 0.329},
 {'acousticness': 0.0765,
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/6zeeWid2sgw4lap2jV61PZ',
  'danceability': 0.596,
  'duration_ms': 229320,
  'energy': 0.552,
  'id': '6zeeWid2sgw4lap2jV61PZ',
  'instrumentalness': 0.334,
  'key': 0,
  'liveness': 0.104,
  'loudness': -10.278,
  'mode': 0,
  'speechiness': 0.097,
  'tempo': 97.949,
  'time_signature': 4,
  'track_href': 'https://api.spotify.com/v1/tracks/6zeeWid2sgw4lap2jV61PZ',
  'type': 'audio_features',
  'uri': 'spotify:track:6zeeWid2sgw4lap2jV61PZ',
  'valence': 0.112},
 {'acousticness': 0.354,
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/7dt6x5M1jzdTEt8oCbisTK',
  'danceability': 0.68,
  'duration_ms': 231267,
  'energy': 0.563,
  'id': '7dt6x5M1jzdTEt8oCbisTK',
  'instrumentalnes

In [30]:
#separate the dictionary into lists
afkeys = [list(x.keys()) for x in audiofeatures]
afvalues = [list(x.values()) for x in audiofeatures]

#create a dataframe for audio features
af = pd.DataFrame(afvalues, columns=afkeys[0])
#
tdata = pd.DataFrame(list(zip(tracks, artists)),columns=["tracks", "artists"])

In [38]:
#combine the two dataframes 
data = pd.concat([tdata, af], axis=1, sort=False)
data

,tracks,artists,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,7 rings,Ariana Grande,0.771,0.320,1,-10.752,0,0.3160,0.60100,0.000000,0.0885,0.312,139.844,audio_features,6ocbgoVGwYJhOv1GgI9NsF,spotify:track:6ocbgoVGwYJhOv1GgI9NsF,https://api.spotify.com/v1/tracks/6ocbgoVGwYJh...,https://api.spotify.com/v1/audio-analysis/6ocb...,178627,4
1,"break up with your girlfriend, i'm bored",Ariana Grande,0.726,0.545,5,-5.489,0,0.1020,0.03780,0.000000,0.1080,0.348,169.991,audio_features,4kV4N9D1iKVxx1KLvtTpjS,spotify:track:4kV4N9D1iKVxx1KLvtTpjS,https://api.spotify.com/v1/tracks/4kV4N9D1iKVx...,https://api.spotify.com/v1/audio-analysis/4kV4...,190440,4
2,MIDDLE CHILD,J. Cole,0.837,0.364,8,-11.713,1,0.2760,0.14900,0.000000,0.2710,0.463,123.984,audio_features,2JvzF1RMd7lE3KmFlsyZD8,spotify:track:2JvzF1RMd7lE3KmFlsyZD8,https://api.spotify.com/v1/tracks/2JvzF1RMd7lE...,https://api.spotify.com/v1/audio-analysis/2Jvz...,213594,4
3,Wow.,Post Malone,0.833,0.539,11,-7.399,0,0.1780,0.16300,0.000002,0.1010,0.385,99.947,audio_features,6MWtB6iiXyIwun0YzU6DFP,spotify:track:6MWtB6iiXyIwun0YzU6DFP,https://api.spotify.com/v1/tracks/6MWtB6iiXyIw...,https://api.spotify.com/v1/audio-analysis/6MWt...,149520,4
4,Sunflower - Spider-Man: Into the Spider-Verse,Various Artists,0.760,0.479,2,-5.574,1,0.0466,0.55600,0.000000,0.0703,0.913,89.911,audio_features,3KkXRkHbMCARz0aVfEt68P,spotify:track:3KkXRkHbMCARz0aVfEt68P,https://api.spotify.com/v1/tracks/3KkXRkHbMCAR...,https://api.spotify.com/v1/audio-analysis/3KkX...,158040,4
5,Robbery,Juice WRLD,0.708,0.691,2,-5.179,1,0.0442,0.34200,0.000000,0.2220,0.535,79.990,audio_features,44ShclXTOOnYtt5CnCDeGV,spotify:track:44ShclXTOOnYtt5CnCDeGV,https://api.spotify.com/v1/tracks/44ShclXTOOnY...,https://api.spotify.com/v1/audio-analysis/44Sh...,240527,4
6,Murder On My Mind,YNW Melly,0.759,0.730,0,-7.985,0,0.0516,0.14500,0.000003,0.1100,0.740,115.007,audio_features,7eBqSVxrzQZtK2mmgRG6lC,spotify:track:7eBqSVxrzQZtK2mmgRG6lC,https://api.spotify.com/v1/tracks/7eBqSVxrzQZt...,https://api.spotify.com/v1/audio-analysis/7eBq...,268434,4
7,Going Bad (feat. Drake),Meek Mill,0.889,0.496,4,-6.365,0,0.0905,0.25900,0.000000,0.2520,0.544,86.003,audio_features,2IRZnDFmlqMuOrYOLnZZyc,spotify:track:2IRZnDFmlqMuOrYOLnZZyc,https://api.spotify.com/v1/tracks/2IRZnDFmlqMu...,https://api.spotify.com/v1/audio-analysis/2IRZ...,180522,4
8,a lot,21 Savage,0.837,0.636,1,-7.643,1,0.0860,0.03950,0.001250,0.3420,0.274,145.972,audio_features,2t8yVaLvJ0RenpXUIAC52d,spotify:track:2t8yVaLvJ0RenpXUIAC52d,https://api.spotify.com/v1/tracks/2t8yVaLvJ0Re...,https://api.spotify.com/v1/audio-analysis/2t8y...,288624,4
9,Thotiana,Blueface,0.906,0.382,10,-12.890,0,0.2690,0.18000,0.000000,0.1130,0.391,104.025,audio_features,3oh6SCCeLuXhFpEyepla6G,spotify:track:3oh6SCCeLuXhFpEyepla6G,https://api.spotify.com/v1/tracks/3oh6SCCeLuXh...,https://api.spotify.com/v1/audio-analysis/3oh6...,129264,4
